In [1]:
library(tidyverse)
library(rawrr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(pracma)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘pracma’


The following object is masked from ‘package:rawrr’:

    tic


The following object is masked from ‘package:purrr’:

    cross




library(foreach)
library(doParallel)

In [2]:
folder_path<-"13_Oct_iter1_v2/"
f2<-str_sub(folder_path,end=-2)

In [3]:
names<-c()
for (i in  list.files(path = folder_path)){
    if (endsWith(i,".raw")){
        names<-append(names,i)
    }
}

extract<-function(name){
    rawfile<-paste(folder_path,name, sep="/")
    index<-rawrr::readIndex(rawfile = rawfile)
    
    scans<-unique(index$scanType)

    megatable <- data.frame(matrix(ncol = 3, nrow = 0))
    x <- c("rt", "tic","scan_type")
    colnames(megatable) <- x
    megatable$scan_type<-as.character(megatable$scan_type)

    for (i in scans){

      subi<-rawrr::readIndex(rawfile = rawfile) |>
        subset(scanType == i)
      listscan<-subi$scan
      S<- rawrr::readSpectrum(rawfile = rawfile, scan = listscan)
      for (j in 1:length(listscan)) {
        datos<-S[[j]]
        ticn<-datos$TIC
        sec<-datos$rtinseconds
        megatable<-megatable %>% add_row(rt = sec, tic = ticn,scan_type = i)
      }

    }
    megatable$file<-name
    return(megatable)
}

In [4]:
names

[1] "0_28_63_mM_1.raw"             "0_28_63_mM_2.raw"            
 [3] "0_28_63_mM_3.raw"             "0_28_63_mM_4.raw"            
 [5] "0_28_63_mM_5.raw"             "0_28_63_mM_6.raw"            
 [7] "1_27_72_mM_1.raw"             "1_27_72_mM_2.raw"            
 [9] "1_27_72_mM_3.raw"             "1_27_72_mM_4.raw"            
[11] "1_27_72_mM_5.raw"             "1_27_72_mM_6.raw"            
[13] "1_93_60_mM_1.raw"             "1_93_60_mM_2.raw"            
[15] "1_93_60_mM_3.raw"             "1_93_60_mM_4.raw"            
[17] "1_93_60_mM_5.raw"             "1_93_60_mM_6.raw"            
[19] "2_19_42_mM_1.raw"             "2_19_42_mM_2.raw"            
[21] "2_19_42_mM_3.raw"             "2_19_42_mM_4.raw"            
[23] "2_19_42_mM_5.raw"             "2_19_42_mM_6.raw"            
[25] "2_50_72_mM_1.raw"             "2_50_72_mM_2.raw"            
[27] "2_50_72_mM_3.raw"             "2_50_72_mM_4.raw"            
[29] "2_50_72_mM_5.raw"             "2_50_72_mM_6.raw"            
[31] "3_09_54_mM_1.raw"             "3_09_54_mM_2.raw"            
[33] "3_09_54_mM_3.raw"             "3_09_54_mM_4.raw"            
[35] "3_09_54_mM_5.raw"             "3_09_54_mM_6.raw"            
[37] "3_27_114_mM_1.raw"            "3_27_114_mM_2.raw"           
[39] "3_27_114_mM_3.raw"            "3_27_114_mM_4.raw"           
[41] "3_27_114_mM_5.raw"            "3_27_114_mM_6.raw"           
[43] "Control_M9_no_bacteria_1.raw" "Control_M9_no_bacteria_2.raw"
[45] "Control_M9_no_bacteria_3.raw" "Control_M9_no_bacteria_4.raw"
[47] "Control_M9_no_bacteria_5.raw" "Control_M9_no_bacteria_6.raw"
[49] "Reference_M9_1.raw"           "Reference_M9_2.raw"          
[51] "Reference_M9_3.raw"           "Reference_M9_4.raw"          
[53] "Reference_M9_5.raw"           "Reference_M9_6.raw"

#setup parallel backend to use many processors
cores=detectCores()
cl <- makeCluster(cores[1]-1) #not to overload your computer
registerDoParallel(cl)

hypertable <- foreach(i=1:length(names), .combine=rbind) %dopar% {
   extract(names[i])
}
#stop cluster
stopCluster(cl)

In [5]:
probar<-list()

for (j in 1:length(names)){
  probar[[j]] <- extract(names[j])
}

In [6]:
hypertable<-do.call("rbind", probar)

write_csv(hypertable, paste(f2,"_hypertable.csv",sep=""))
#sample<-subset(hypertable, scan_type=="+ c ESI SRM ms2 76.039 [30.189-30.191, 44.011-44.013]")

#ggplot(sample,aes(x=rt,y=tic,color=file))+geom_line()

In [7]:
#trapz <- function(y, x) { #home-made version of trapezoid rule
#  n <- length(y)
#  dx <- diff(x)
#  (dx * (y[-1] + y[-n]) / 2) %>% sum
#}

integral <- hypertable %>%
  group_by(scan_type, file) %>%
  summarise(integral = pracma::trapz(rt,tic)) %>%
  ungroup()


mati <- integral %>%
  pivot_wider(id_cols = file, names_from = scan_type, values_from = integral)


write_csv(mati, paste(f2,"_matrix.csv",sep=""))

`summarise()` has grouped output by 'scan_type'. You can override using the
`.groups` argument.


In [8]:
trapz

function (x, y) 
{
    if (missing(y)) {
        if (length(x) == 0) 
            return(0)
        y <- x
        x <- seq(along = x)
    }
    if (length(x) == 0 && length(y) == 0) 
        return(0)
    if (!(is.numeric(x) || is.complex(x)) || !(is.numeric(y) || 
        is.complex(y))) 
        stop("Arguments 'x' and 'y' must be real or complex vectors.")
    m <- length(x)
    if (length(y) != m) 
        stop("Arguments 'x', 'y' must be vectors of the same length.")
    if (m <= 1) 
        return(0)
    xp <- c(x, x[m:1])
    yp <- c(numeric(m), y[m:1])
    n <- 2 * m
    p1 <- sum(xp[1:(n - 1)] * yp[2:n]) + xp[n] * yp[1]
    p2 <- sum(xp[2:n] * yp[1:(n - 1)]) + xp[1] * yp[n]
    return(0.5 * (p1 - p2))
}
<bytecode: 0x564fab29bb90>
<environment: namespace:pracma>